In [9]:
import EarthModel
import numpy as np
import quaternion
import math

ModuleNotFoundError: No module named 'quaternion'

In [2]:
def FormQuaternionFromEulerAngles(EulerAttitude):

    cRoll    = np.cos(
         EulerAttitude['NAV_FRAME_EULER_ROLL'] / 2.0);
    sRoll    = np.sin(
         EulerAttitude['NAV_FRAME_EULER_ROLL'] / 2.0);
    cPitch   = np.cos(
         EulerAttitude['NAV_FRAME_EULER_PITCH'] / 2.0);
    sPitch   = np.sin(
         EulerAttitude['NAV_FRAME_EULER_PITCH'] / 2.0);
    cHeading = np.cos(
         EulerAttitude['NAV_FRAME_EULER_HEADING'] / 2.0);
    sHeading = np.sin(
         EulerAttitude['NAV_FRAME_EULER_HEADING'] / 2.0);


    result = np.quaternion(cRoll*cPitch*cHeading + sRoll*sPitch*sHeading,
      sRoll*cPitch*cHeading - cRoll*sPitch*sHeading,
      cRoll*sPitch*cHeading + sRoll*cPitch*sHeading,
      cRoll*cPitch*sHeading - sRoll*sPitch*cHeading)
    return result

In [3]:
def FormDCMFromQuaternion(q):

   result = np.identity(3)
   ss = q.w * q.w;
   ii = q.x * q.x;
   jj = q.y * q.y;
   kk = q.z * q.z;

   jk = 2.0 * q.y * q.z;
   ij = 2.0 * q.x * q.y;
   ik = 2.0 * q.x * q.z;
   si = 2.0 * q.w * q.x;
   sj = 2.0 * q.w * q.y;
   sk = 2.0 * q.w * q.z;


   result[0][0] = ss + ii - jj - kk;
   result[0][1] = ij - sk;
   result[0][2] = ik + sj;
   result[1][0] = ij + sk;
   result[1][1] = ss - ii + jj - kk;
   result[1][2] = jk - si;
   result[2][0] = ik - sj;
   result[2][1] = jk + si;
   result[2][2] = ss - ii - jj + kk;
   
   return result

In [4]:
def ExtractEulerAnglesFromDCM(DCM):

   result = {'NAV_FRAME_EULER_ROLL' : np.arctan2(DCM[2][1],DCM[2][2]),
             'NAV_FRAME_EULER_PITCH' : -np.arcsin(DCM[2][0]),
             'NAV_FRAME_EULER_HEADING' : np.arctan2(DCM[1][0],DCM[0][0])
             }
   return result

In [5]:
def FormQuaternionUpdate(Increment):

   
   # Create a quaternion that represents a small Euler angle update.

   # Quaternion update defined as Ac + As*SigmaX i + As*SigmaY j +
   # As*SigmaZ k as in Weston
   result = np.quaternion(1,0,0,0)
   IncrementMagnitude = np.sqrt(
                  Increment[0] * Increment[0] +
                  Increment[1] * Increment[1] +
                  Increment[2] * Increment[2])


   if (IncrementMagnitude != 0.0):
      As = np.sin(IncrementMagnitude / 2.0)
      Ac = np.cos(IncrementMagnitude / 2.0)
      As /= IncrementMagnitude;

      result.w = Ac;
      result.x = As * Increment[0]
      result[2] = As * Increment[1]
      result.z = As * Increment[2]
   else:
      result.w = 1.0
      result.x = 0.0
      result.y = 0.0
      result.z = 0.0
      
   return result

In [6]:
testEuler1 = {
    'NAV_FRAME_EULER_ROLL':0,
    'NAV_FRAME_EULER_PITCH':90*np.pi/180.0,
    'NAV_FRAME_EULER_HEADING':0.0
}

In [7]:
print(testEuler1)

{'NAV_FRAME_EULER_ROLL': 0, 'NAV_FRAME_EULER_PITCH': 1.5707963267948966, 'NAV_FRAME_EULER_HEADING': 0.0}


In [8]:
testQ1 = FormQuaternionFromEulerAngles(testEuler1)

AttributeError: module 'numpy' has no attribute 'quaternion'